In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
#Load Data
path = "data.csv"
df = pd.read_csv(path)

df

In [ ]:
#Get Cells
df = df[(df['Cell'] == 'Cell1') | (df['Cell'] == 'Cell4')]
df = df.reset_index(drop=True)

df

In [ ]:
#Get features
features = list(df.columns)[4:6]
data = df[features]

data

In [ ]:
def DistanceMatrix(center,x):
    distanceMatrix = cdist(x, center, 'euclidean').T
    return distanceMatrix

def HardPartition(center,x):
    n,kk=x.shape
    result=np.zeros((n,1))
    d=DistanceMatrix(center,x)
    mind=d.min(axis=0)
    for i in range(n):
        result[i] =(d[:,[i]]==mind[i]).argmax()
    return result

In [ ]:
centers_file = open("center.pkl",'rb')
centers = pickle.load(centers_file)

labels_file = open("label.pkl",'rb')
labels = pickle.load(labels_file)

In [ ]:
result = HardPartition(centers, data)

result

In [ ]:
df['clusters'] = result

In [ ]:
#Get prediction value
def pred_t_remain(row):
  return int(labels[int(row['clusters'])])

df['pred_t_remain'] = df.apply (pred_t_remain, axis=1)

df

In [ ]:
#Evaluation
rmse = mean_squared_error(df['t_remain'], df['pred_t_remain'], squared=False)
mae = mean_absolute_error(df['t_remain'], df['pred_t_remain'])
r2 = r2_score(df['t_remain'], df['pred_t_remain'])

print(rmse)
print(mae)
print(r2)

In [ ]:
#Convert remaining time charging to minutes
def t_remain_to_min(row):
  return int(row['t_remain']/60)

def center_t_to_min(row):
  return int(row['pred_t_remain']/60)

df['t_remain_to_min'] = df.apply (t_remain_to_min, axis=1)
df['center_t_to_min'] = df.apply (center_t_to_min, axis=1)

df

In [ ]:
#Create prediction error column
df['t_diff'] = abs(df['center_t_to_min'] - df['t_remain_to_min'])

In [ ]:
#Visualize
tmp = df
fig, ax = plt.subplots(figsize=(8,4))
tmp.groupby(['t_remain_to_min']).mean()['t_diff'].plot(ax=ax)
ax.set_xlabel('Time Remaining (minutes)')
ax.set_ylabel('Prediction Error (minutes)')
ax.invert_xaxis()